### In the code given below we have fine tuned Imdb datset using mistral 7B Quantized model incorporating UNSLOTH for faster fine tuning. The approach which we used is SFT( supervised fined tuning ) which is available from a library called TRL (Transformer reinforcement learning) and used FAST LoRa weigths.

In [2]:
### How does LoRA helps in Fine-Tuning? 
### Ans: A low-rank matrix is added to the weights of the pre-trained model. This low-rank matrix is learned during the fine-tuning process and is used to adapt the model to the specific task or dataset.

In [1]:
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-kcfersvp/unsloth_d513997eac3c4b7f8a71db386f235470
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-kcfersvp/unsloth_d513997eac3c4b7f8a71db386f235470
  Resolved https://github.com/unslothai/unsloth.git to commit ce523ed15cf56a6cd05fa4646153e7688ec44c8a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install "git+https://github.com/huggingface/transformers.git"

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-50hxrqen
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-50hxrqen
  Resolved https://github.com/huggingface/transformers.git to commit 8c12690cecbb97e187861e386f7a0ac790e4236c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 19.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.41.0.dev0-py3-none-any.whl size=9012708 sha256=7b6a954e734772cfe7d39175df4620c54da0a50d9a9069799ba9240cb0ba4936
  Stored in directory: /tmp/pip-ephem-wheel-cache-jcue1vgh/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      

In [3]:
!pip install trl

In [6]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
import torch
from transformers import TrainingArguments

In [7]:
from datasets import load_dataset

In [8]:
max_seq_len = 2048

In [9]:
dataset = load_dataset("imdb",split="train")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [13]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length = max_seq_len,
    dtype=None,
    load_in_4bit=True
)

Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


==((====))==  Unsloth: Fast Mistral patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [14]:
# GET MODEL PATCHING AND ADD FAST LORA (Low-Rank Additive Regularization for Adaptation) WEIGHTS
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_len
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [16]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_len,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "unsloth-test",
        optim = "adamw_8bit",
        seed = 3407,
    ),
)
trainer.train()

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 25,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.605500
2,2.343800
3,2.341800
4,2.476600
5,2.539100
6,2.685500
7,2.353500
8,2.371100
9,2.222700
10,2.541000


TrainOutput(global_step=60, training_loss=2.4264322916666665, metrics={'train_runtime': 743.1355, 'train_samples_per_second': 0.646, 'train_steps_per_second': 0.081, 'total_flos': 9642624560529408.0, 'train_loss': 2.4264322916666665, 'epoch': 0.0192})

In [17]:
inputs = tokenizer(
    [
      "I really liked the movie because it shows emotions and talks humanity."
    ],
    return_tensors="pt",
).to("cuda")

In [18]:
inputs

{'input_ids': tensor([[    1,   315,  1528,  8232,   272,  5994,  1096,   378,  4370, 13855,
           304, 15066, 17676, 28723]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [19]:
outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [20]:
outputs

tensor([[    1,   315,  1528,  8232,   272,  5994,  1096,   378,  4370, 13855,
           304, 15066, 17676, 28723,   661,   349,   264,  1215,  1179,  5994,
         28723,   315,  1528,  8232,   272,  5994,  1096,   378,  4370, 13855,
           304, 15066, 17676, 28723,   661,   349,   264,  1215,  1179,  5994,
         28723,   315,  1528,  8232,   272,  5994,  1096,   378,  4370, 13855,
           304, 15066, 17676, 28723,   661,   349,   264,  1215,  1179,  5994,
         28723,   315,  1528,  8232,   272,  5994,  1096,   378,  4370, 13855,
           304, 15066, 17676, 28723,   661,   349,   264,  1215,  1179,  5994,
         28723,   315,  1528,  8232,   272,  5994,  1096,   378,  4370, 13855,
           304, 15066, 17676, 28723,   661,   349,   264,  1215,  1179,  5994,
         28723,   315,  1528,  8232,   272,  5994,  1096,   378,  4370, 13855,
           304, 15066, 17676, 28723,   661,   349,   264,  1215,  1179,  5994,
         28723,   315,  1528,  8232,   272,  5994,  

In [21]:
tokenizer.batch_decode(outputs)

['<s> I really liked the movie because it shows emotions and talks humanity. It is a very good movie. I really liked the movie because it shows emotions and talks humanity. It is a very good movie. I really liked the movie because it shows emotions and talks humanity. It is a very good movie. I really liked the movie because it shows emotions and talks humanity. It is a very good movie. I really liked the movie because it shows emotions and talks humanity. It is a very good movie. I really liked the movie because it shows emotions and talks humanity. It is a very good movie. I really liked the movie because it shows emotions and talks humanity. It is a very good movie. I']